In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'fourier'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_fourier_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,54044.85,3.34,-2.58,-0.52,0.98,0.15,-0.09,-0.64,0.09,-0.47,...,-0.02,-0.01,-0.06,0.04,0.01,0.01,-0.0,-0.00,-0.0,0.00
3,3.34,84098.62,-0.41,0.05,2.02,1.13,0.24,0.81,0.55,0.52,...,-0.07,-0.01,-0.02,0.01,0.01,0.00,-0.0,-0.00,-0.0,0.00
4,-2.58,-0.41,50919.68,-0.64,-0.46,-0.10,1.12,-0.36,0.23,0.54,...,-0.09,0.04,0.04,-0.01,-0.00,-0.01,0.0,0.00,0.0,-0.00
5,-0.52,0.05,-0.64,26792.70,-0.53,0.26,-0.50,-0.39,-0.47,-0.46,...,0.04,0.01,0.02,0.00,-0.00,-0.00,-0.0,-0.00,0.0,0.00
6,0.98,2.02,-0.46,-0.53,16298.42,-0.07,-0.07,-0.40,-0.32,-0.24,...,-0.02,0.02,0.02,-0.01,0.00,-0.00,0.0,-0.00,-0.0,-0.00
7,0.15,1.13,-0.10,0.26,-0.07,10349.58,-0.09,-0.01,0.41,0.13,...,0.01,0.05,-0.00,0.02,-0.00,0.00,0.0,0.00,-0.0,-0.00
8,-0.09,0.24,1.12,-0.50,-0.07,-0.09,7450.95,-0.11,0.16,0.30,...,-0.01,-0.03,-0.00,0.00,-0.00,-0.00,0.0,-0.00,-0.0,-0.00
9,-0.64,0.81,-0.36,-0.39,-0.40,-0.01,-0.11,9192.80,-0.10,0.19,...,0.03,0.01,-0.02,0.00,-0.00,0.00,-0.0,0.00,-0.0,-0.00
10,0.09,0.55,0.23,-0.47,-0.32,0.41,0.16,-0.10,5784.92,-0.21,...,0.01,0.00,0.00,-0.01,-0.00,0.00,0.0,-0.00,-0.0,0.00
11,-0.47,0.52,0.54,-0.46,-0.24,0.13,0.30,0.19,-0.21,5910.34,...,0.00,-0.01,-0.00,0.01,-0.00,0.00,-0.0,0.00,-0.0,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,0.00005,-0.00005,-0.00001,0.00003,0.00001,-0.00000,-0.00003,0.00000,-0.00003,...,-0.00001,-0.00000,-0.00004,0.00003,0.00001,0.00001,-0.00000,-0.00002,-0.00001,0.00000
3,0.00005,1.00000,-0.00001,0.00000,0.00005,0.00004,0.00001,0.00003,0.00003,0.00002,...,-0.00002,-0.00000,-0.00001,0.00001,0.00001,0.00000,-0.00000,-0.00000,-0.00000,0.00000
4,-0.00005,-0.00001,1.00000,-0.00002,-0.00002,-0.00000,0.00006,-0.00002,0.00001,0.00003,...,-0.00004,0.00001,0.00003,-0.00001,-0.00000,-0.00002,0.00001,0.00002,0.00001,-0.00000
5,-0.00001,0.00000,-0.00002,1.00000,-0.00003,0.00002,-0.00004,-0.00002,-0.00004,-0.00004,...,0.00002,0.00001,0.00002,0.00000,-0.00000,-0.00001,-0.00000,-0.00000,0.00003,0.00001
6,0.00003,0.00005,-0.00002,-0.00003,1.00000,-0.00001,-0.00001,-0.00003,-0.00003,-0.00002,...,-0.00001,0.00001,0.00003,-0.00002,0.00001,-0.00000,0.00001,-0.00001,-0.00000,-0.00000
7,0.00001,0.00004,-0.00000,0.00002,-0.00001,1.00000,-0.00001,-0.00000,0.00005,0.00002,...,0.00001,0.00003,-0.00000,0.00003,-0.00002,0.00000,0.00001,0.00001,-0.00001,-0.00001
8,-0.00000,0.00001,0.00006,-0.00004,-0.00001,-0.00001,1.00000,-0.00001,0.00002,0.00004,...,-0.00001,-0.00002,-0.00000,0.00000,-0.00000,-0.00001,0.00001,-0.00000,-0.00000,-0.00001
9,-0.00003,0.00003,-0.00002,-0.00002,-0.00003,-0.00000,-0.00001,1.00000,-0.00001,0.00003,...,0.00003,0.00001,-0.00003,0.00000,-0.00001,0.00001,-0.00001,0.00002,-0.00000,-0.00000
10,0.00000,0.00003,0.00001,-0.00004,-0.00003,0.00005,0.00002,-0.00001,1.00000,-0.00004,...,0.00002,0.00000,0.00001,-0.00003,-0.00000,0.00002,0.00002,-0.00001,-0.00001,0.00001
11,-0.00003,0.00002,0.00003,-0.00004,-0.00002,0.00002,0.00004,0.00003,-0.00004,1.00000,...,0.00001,-0.00000,-0.00001,0.00003,-0.00000,0.00002,-0.00001,0.00001,-0.00000,-0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

8.23268446182307

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.21346451e+08 9.19271320e+07 8.02082163e+07 2.23691073e+07
 8.27516729e+06 3.35065281e+06 2.62274846e+06 1.72308618e+06
 1.10827460e+06 1.00459188e+06 5.01451596e+05 2.98120350e+05
 2.38871473e+05 6.93452061e+04 5.29677794e+04 3.82063681e+04
 2.72937846e+04 1.05127724e+04 4.29201173e+03 3.58438887e+03
 1.67275741e+03 1.33268155e+03 3.95499083e+02 1.69244161e+02
 3.87343415e+01 2.76726252e+01 1.48468831e+00 5.23477320e-01
 2.02876666e-02 4.71465896e-03 1.60340123e-04 4.36528713e-23]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-3.421691e-02,9.988652e-01,-2.987335e-02,-1.091561e-02,-0.006156,-0.003821,-0.002738,-0.003384,-0.002114,-0.002161,...,-0.000043,-0.000085,-0.000013,-0.000012,-0.000002,-0.000003,-3.356825e-07,-1.740894e-07,-3.584233e-08,-6.336900e-09
1,9.668275e-01,2.525652e-02,-2.533994e-01,-1.601574e-02,-0.008002,-0.004802,-0.003404,-0.004244,-0.002615,-0.002686,...,-0.000052,-0.000105,-0.000017,-0.000014,-0.000002,-0.000003,-4.182337e-07,-2.820238e-07,-5.850736e-08,-7.949068e-09
2,2.515538e-01,3.698506e-02,9.664109e-01,-3.058999e-02,-0.014808,-0.008777,-0.006158,-0.007731,-0.004751,-0.004854,...,-0.000096,-0.000188,-0.000028,-0.000027,-0.000004,-0.000006,-6.971053e-07,-3.534381e-07,-7.440825e-08,-1.941526e-08
3,2.208968e-02,1.199193e-02,2.441257e-02,9.984014e-01,-0.034700,-0.016039,-0.010648,-0.013753,-0.007995,-0.008186,...,-0.000151,-0.000305,-0.000046,-0.000043,-0.000006,-0.000010,-1.207875e-06,-6.420196e-07,-7.093934e-08,-2.554731e-08
4,1.128449e-02,6.851014e-03,1.216749e-02,3.242840e-02,0.997584,-0.038635,-0.020675,-0.029855,-0.014492,-0.014898,...,-0.000255,-0.000506,-0.000076,-0.000072,-0.000010,-0.000016,-1.874985e-06,-1.091397e-06,-2.186754e-07,-5.379817e-08
5,5.924907e-03,3.717011e-03,6.328769e-03,1.346942e-02,0.031146,0.986482,-0.047664,-0.146796,-0.025309,-0.026487,...,-0.000347,-0.000688,-0.000106,-0.000097,-0.000014,-0.000022,-2.625930e-06,-1.386474e-06,-3.046396e-07,-7.867578e-08
6,6.527270e-03,4.104919e-03,6.953390e-03,1.442260e-02,0.030314,0.137285,-0.098976,0.982620,-0.041408,-0.043572,...,-0.000488,-0.000979,-0.000152,-0.000139,-0.000019,-0.000031,-3.866332e-06,-1.873709e-06,-4.098882e-07,-1.012081e-07
7,5.244390e-03,3.328515e-03,5.555655e-03,1.123940e-02,0.021368,0.053655,0.986680,0.082308,-0.080537,-0.088715,...,-0.000609,-0.001217,-0.000186,-0.000171,-0.000024,-0.000039,-4.546011e-06,-2.495265e-06,-5.115802e-07,-1.450896e-07
8,1.550604e-03,9.812737e-04,1.631013e-03,3.234665e-03,0.005805,0.011735,0.028879,0.015028,-0.536635,0.842740,...,-0.000280,-0.000560,-0.000087,-0.000076,-0.000011,-0.000017,-2.351360e-06,-9.976310e-07,-2.029951e-07,-7.633084e-08
9,6.269607e-03,3.989667e-03,6.649342e-03,1.310488e-02,0.023324,0.045556,0.102239,0.057714,0.834740,0.522908,...,-0.001252,-0.002511,-0.000384,-0.000355,-0.000049,-0.000078,-9.535155e-06,-5.133701e-06,-1.101542e-06,-2.502889e-07


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.034216910657253,
 0.9747434822673958,
 0.033589115420712745,
 0.0015985836191346747,
 0.0024161009548367973,
 0.013518395231349123,
 1.098976155190479,
 0.9176917560876366,
 1.5366348277562971,
 0.477091605823625,
 0.011170973040357746,
 1.2474753555558755,
 1.04880878149658,
 1.074611030935442,
 1.0852732829974847,
 1.043642306632277,
 1.0366574389279768,
 1.066077583748949,
 1.0313124265747213,
 0.9898490749312472,
 0.9930355598431526,
 1.0560896216397644,
 0.016556211257455655,
 0.9582149896876494,
 0.1436766706919027,
 0.14785850597845562,
 1.192170020587096,
 0.8117532470975974,
 0.027949044744072116,
 0.03255789970840339,
 0.03675517544961038,
 0.031037234277283665]